### 映画レビューデータセットを読み込み

In [1]:
# データを読み込み
import pyprind
import pandas as pd
import os
import io

pbar = pyprind.ProgBar(50000)
labels = {'pos':1, 'neg':0}
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = './aclImdb/%s/%s' % (s, l)
        for file in os.listdir(path):
            with io.open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
                df = df.append([[txt, labels[l]]], ignore_index=True)
                pbar.update()
                
df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:56


In [2]:
# データをシャッフルして保存
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('./movie_data.csv', index=False, encoding='utf-8')

In [3]:
df = pd.read_csv('./movie_data.csv')
df.head(3)

,review,sentiment
0,"Like NIGHT STALKER and then X-FILES, the show ...",1
1,It's been over 30 years now but I still rememb...,0
2,I was interested in seeing this movie because ...,0


### レビューに含まれる単語を特徴ベクトルに変換

In [4]:
# 単語を特徴ベクトルに変換するサンプル
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array(['The sun is shining',
                 'The weather is sweet',
                 'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)
print(count.vocabulary_)
print(bag.toarray())

{u'and': 0, u'sun': 4, u'is': 1, u'two': 7, u'one': 2, u'weather': 8, u'sweet': 5, u'the': 6, u'shining': 3}
[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


### TF-IDF

Term Frequency … ある文書中のある単語の出現頻度<br>
Inverse Document Frequency … 文書全体のうち、ある単語を含んでいる文書の割合 の「逆数」, 該当する文書が少ないほど大きくなる<br>
$$
tfidf(t, d) = tf(t, d) \times idf(t, d) = \frac{n_t}{\sum_k n_{k, d}} \times log \frac{1 + |D|}{1 + |d : d \ni t_i|}
$$

In [5]:
# scikit-learnのクラスでtfidf変換するサンプル
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True) # アウトプットの特徴ベクトルをL2正規化する
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[ 0.    0.43  0.    0.56  0.56  0.    0.43  0.    0.  ]
 [ 0.    0.43  0.    0.    0.    0.56  0.43  0.    0.56]
 [ 0.5   0.45  0.5   0.19  0.19  0.19  0.3   0.25  0.19]]


### データを綺麗にする

In [6]:
# データのクレンジング
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) # マークアップの削除
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text) # 顔文字を検出
    text = re.sub('[\W]+', ' ', text.lower()) + ''.join(emoticons).replace('-', '') # 空白文字をまとめ、顔文字を末尾に付け足す
    return text

In [7]:
# タグが消え、顔文字はまとめて後ろに付与される
preprocessor(u'</a>This :) is :( a test :-)!')

u'this is a test :):(:)'

In [8]:
# 実装したpreprocessorで変換
df['review'] = df.review.apply(preprocessor)

### 文章を単語にする（トークン化）

In [9]:
# スペースで区切る単純なトークン化
def tokenizer(text):
    return text.split()

In [10]:
tokenizer(u'runners like running and thus they run')

[u'runners', u'like', u'running', u'and', u'thus', u'they', u'run']

In [11]:
# ワードステミングをしてからトークン化
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [12]:
# running -> run とか
tokenizer_porter(u'runners like running and thus they run')

[u'runner', u'like', u'run', u'and', u'thu', u'they', u'run']

### ストップワード（ありふれた単語の除去）

In [13]:
# TFIDFでは頻出単語の重みを減らしているので不要のはず
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/yamada/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
[w for w in tokenizer_porter(u'a runner likes running and runs a lot') if w not in stop]

[u'runner', u'like', u'run', u'run', u'lot']

### ロジスティック回帰モデルでトレーニング

In [15]:
# データをトレーニング用とテスト用に分割
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [16]:
# グリッドサーチで最適なパラメータを探索
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    strip_accents=None,
    lowercase=False,
    preprocessor=None,
    tokenizer=tokenizer,
    stop_words=None,
    ngram_range=(1, 1)
)

# 非常に時間がかかるため、省略
"""
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]
"""
param_grid = [{
        'clf__penalty': ['l2'],
        'clf__C': [1.0, 10.0]
    }]
lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [17]:
# トレーニングデータ全体でfit
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   28.8s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('vect', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=Tru...nalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid=[{'clf__penalty': ['l2'], 'clf__C': [1.0, 10.0]}],
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=1)

In [18]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

Best parameter set: {'clf__penalty': 'l2', 'clf__C': 10.0} 
CV Accuracy: 0.896


In [19]:
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Test Accuracy: 0.897


### 大規模なデータをオンラインで処理する例（アウトオブコア学習）

In [20]:
import numpy as np
import re
import io
from nltk.corpus import stopwords

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with io.open(path, 'r', encoding='utf-8') as csv:
        next(csv) # ヘッダ行をとばす
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label # yield-関数の処理を 一旦停止し値を返し、再開

#### （参考）stream_docsについての説明
イテレータとは…<br>
イテレータとは、繰り返しを抽象化したものである<br>
（リストなどの）構造を持ったオブジェクトの各要素を繰り返し参照するために利用される<br>
元となる「構造を持ったオブジェクト」を引数にしてiter()関数を呼ぶことで生成できる<br>
（内部的には構造を持ったオブジェクトに対してiter()メソッドを呼んでいる）<br>
イテレータに対してnext()メソッドを呼ぶと、元となる「構造を持ったオブジェクト」の値を一つずつ返す<br>
アクセスすべき要素がないイテレータに対してnext()メソッドを呼ぶとStopIterationという例外を発生(raise)する<br>
<br>
ジェネレータとは…<br>
ジェネレータとは、値の生成器である<br>
イテレータとは異なり、必ずしも「構造を持ったオブジェクトの要素を繰り返し参照するもの」ではない<br>
ジェネレータに対してnext()メソッドを呼ぶと、何らかの値が返ってくる<br>
次に返すべき要素がないジェネレータに対してnext()メソッドを呼ぶとStopIteration例外を発生するが、イテレータと異なり必ず「最後の要素」があるわけではない（無限に要素を返すジェネレータもある）<br>
ジェネレータの作り方は、ジェネレータ式を使う方法とyield文を使う方法とがある<br>
yieldではreturnと異なり、関数の処理を「一旦停止」し値を返す。
活用シーンとしては、巨大なデータを読み込み、別なプログラムに受け渡すような場合、普通に実行すると受け渡し用のメモリが巨大になるが、<br>
少量ずつデータを読み込み、その都度yieldすればいいので、メモリの使用量はほんの僅かで済む

http://qiita.com/HirofumiYashima/items/3cbf95d54478a87d12f0<br>
http://ailaby.com/yield/

In [21]:
# nextを呼ぶと要素が順番に返るのを試す
next(stream_docs(path='./movie_data.csv'))

(u'"Like NIGHT STALKER and then X-FILES, the show set up a fantastic situation and the main characters had to sort it out. Unlike these, the hero(es) weren\'t left holding an empty bag at the end. They had usually tangible results. It was also made clear that the \'good guys\' were in a dirty profession where they occasionally had to pull some nasty things. Imagination, wit, acting which didn\'t always take itself too seriously ... I miss it. One reason being, I\'m hard pressed to think of too many shows - BANACEK aside - which did as good a job of taking the viewer and grabbing their attention right off the bat. The writers excelled at setting up hugely improbable, if not downright impossible situations which the characters then had to find an explanation to. explanations which often took 90 degree turns into the clearly unexpected yet, for all that, still made sense. Too, I agree with another reviewer that the Anabelle character was somewhat underused, but when she was on screen, it 

In [22]:
# データ全体ではなく、少しずつのデータでモデルを更新するために、指定された個数のドキュメントを返す関数
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

#### Feature Hashing…ハッシュ関数を使って、素性群をM次元ベクトルにする一種の次元圧縮

In [23]:
# Feature Hashingで入力される単語をハッシュ値に変換すれば、未知の単語もハッシュ値として受け取れるので前もって全ての単語を使ったベクトルを
# 用意する必要はない
from sklearn.feature_extraction.text import HashingVectorizer
# Stochastic Gradient Descent 確率的勾配降下法
# 全てのサンプルの誤分類の合計に基づいて重みを更新するのではなく、トレーニングサンプル毎に段階的に重みを更新する
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)
clf = SGDClassifier(loss='log', random_state=1, n_jobs=-1)
doc_strem = stream_docs(path='./movie_data.csv')

In [24]:
import pyprind
pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])
for _ in range(45):
    # データ全体の内、1000個ずつを取り出して学習し、モデルを更新していく
    X_train, y_train = get_minibatch(doc_stream=doc_strem, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    # 前のfitの結果をリセットせず学習を続ける
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:41


In [25]:
X_test, y_test = get_minibatch(doc_stream=doc_strem, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.877


In [26]:
clf = clf.partial_fit(X_test, y_test)

In [27]:
# 作成したモデルを保存
import pickle
import os
dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)
pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=pickle.HIGHEST_PROTOCOL)